In [27]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)


2.20.0


In [28]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0


In [29]:

# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
checkpoint_path = "training_1/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.


Epoch 1/10
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6138 - sparse_categorical_accuracy: 0.4902   
Epoch 1: saving model to training_1/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.1242 - sparse_categorical_accuracy: 0.6760 - val_loss: 0.6864 - val_sparse_categorical_accuracy: 0.7810
Epoch 2/10
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4143 - sparse_categorical_accuracy: 0.8870 
Epoch 2: saving model to training_1/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4173 - sparse_categorical_accuracy: 0.8850 - val_loss: 0.5125 - val_sparse_categorical_accuracy: 0.8460
Epoch 3/10
28/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2782 - sparse_categorical_accuracy: 0.9365 
Epoch 3: saving model to training_1/cp.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2806 - sparse_categorical_accuracy: 0.9330 - val_loss: 0.4728 - val_sparse_categorical_accuracy: 0.8440
Epoch 4/10
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2033 - sparse_cat

In [31]:
os.listdir(checkpoint_dir)


['cp.weights.h5']

In [32]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))


32/32 - 0s - 4ms/step - loss: 2.3623 - sparse_categorical_accuracy: 0.0440
Untrained model, accuracy:  4.40%


In [33]:
#loads the weights 
model.load_weights(checkpoint_path)

# re-evaluate the model 
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:.2f}%'.format(100 * acc))

32/32 - 0s - 1ms/step - loss: 0.4077 - sparse_categorical_accuracy: 0.8710
Restored model, accuracy: 87.10%


In [34]:
 #checkpoint callback options 

In [35]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)



Epoch 5: saving model to training_2/cp-0005.weights.h5

Epoch 10: saving model to training_2/cp-0010.weights.h5

Epoch 15: saving model to training_2/cp-0015.weights.h5

Epoch 20: saving model to training_2/cp-0020.weights.h5

Epoch 25: saving model to training_2/cp-0025.weights.h5

Epoch 30: saving model to training_2/cp-0030.weights.h5

Epoch 35: saving model to training_2/cp-0035.weights.h5

Epoch 40: saving model to training_2/cp-0040.weights.h5

Epoch 45: saving model to training_2/cp-0045.weights.h5

Epoch 50: saving model to training_2/cp-0050.weights.h5


In [36]:
os.listdir(checkpoint_dir)

['cp-0000.weights.h5',
 'cp-0005.weights.h5',
 'cp-0010.weights.h5',
 'cp-0015.weights.h5',
 'cp-0020.weights.h5',
 'cp-0025.weights.h5',
 'cp-0030.weights.h5',
 'cp-0035.weights.h5',
 'cp-0040.weights.h5',
 'cp-0045.weights.h5',
 'cp-0050.weights.h5']

In [37]:
def load_latest_checkpoint(checkpoint_dir):
    latest = max(os.listdir(checkpoint_dir), key=lambda f: int(f.split('-')[1].split('.')[0]))
    return os.path.join(checkpoint_dir, latest)

latest = load_latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.weights.h5'

In [38]:
# Create a new model instance
model = create_model()
print(latest)
# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

training_2\cp-0050.weights.h5
32/32 - 0s - 4ms/step - loss: 0.5034 - sparse_categorical_accuracy: 0.8740
Restored model, accuracy: 87.40%


In [39]:
# Save the weights
os.makedirs('./checkpoints2', exist_ok=True)
model.save_weights('./checkpoints2/my_checkpoint.weights.h5')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints2/my_checkpoint.weights.h5')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - 4ms/step - loss: 0.5034 - sparse_categorical_accuracy: 0.8740
Restored model, accuracy: 87.40%


In [40]:
#create and train a new model instance 
model = create_model() 
model.fit(train_images, train_labels, epochs=5) 

#save the entire model as a '.keras' zip archive 

model.save('my_model.keras')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.1728 - sparse_categorical_accuracy: 0.6590  
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4271 - sparse_categorical_accuracy: 0.8780
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2814 - sparse_categorical_accuracy: 0.9300
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2047 - sparse_categorical_accuracy: 0.9520
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1553 - sparse_categorical_accuracy: 0.9690


In [41]:
new_model = tf.keras.models.load_model('my_model.keras') 

# show the model architecture

new_model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_26 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,221,152 (4.66 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 814,102 (3.11 MB)

In [42]:
#Evalute the restored model 
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - 4ms/step - loss: 0.4428 - sparse_categorical_accuracy: 0.8530
Restored model, accuracy: 85.30%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
(1000, 10)


In [46]:
#create and train a new model instance. 
model = create_model() 
model.fit(train_images, train_labels, epochs=5)

#save the entire model as a SavedModel directory using the Keras API
os.makedirs('saved_model', exist_ok=True)
model.save('saved_model/my_model.keras')

Epoch 1/5


e:\programs\py\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1537 - sparse_categorical_accuracy: 0.6740  
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4122 - sparse_categorical_accuracy: 0.8750
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2898 - sparse_categorical_accuracy: 0.9190
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1952 - sparse_categorical_accuracy: 0.9530
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1562 - sparse_categorical_accuracy: 0.9700


In [49]:
new_model = tf.keras.models.load_model('saved_model/my_model.keras')

# Check its architecture
new_model.summary()


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_34 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,221,152 (4.66 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 814,102 (3.11 MB)

In [50]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - 3ms/step - loss: 0.4463 - sparse_categorical_accuracy: 0.8600
Restored model, accuracy: 86.00%
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
(1000, 10)


In [51]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')


Epoch 1/5


e:\programs\py\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1814 - sparse_categorical_accuracy: 0.6770  
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.4358 - sparse_categorical_accuracy: 0.8820
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2913 - sparse_categorical_accuracy: 0.9290
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2109 - sparse_categorical_accuracy: 0.9510
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1542 - sparse_categorical_accuracy: 0.9670


In [52]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()


Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,052 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [53]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))


32/32 - 0s - 4ms/step - loss: 0.4556 - sparse_categorical_accuracy: 0.8590
Restored model, accuracy: 85.90%
